# Night Shifts

In [163]:
from src.read_csv import parse_csv

import os
import pandas as pd
import copy
import statistics
from itertools import combinations

In [164]:
# Minimum number of people that need to be assigned during night shifts
min_people_night = 2

In [165]:
availabilities = [parse_csv(os.path.join('data', file)) for file in os.listdir('data') if file.endswith('.csv')]

print(f'Number of availabilities: {len(availabilities)}')
print([avail.columns[0] for avail in availabilities])

Number of availabilities: 7
['UNC', 'Michael Bryant', 'Barack Obama', 'Bill Nye', 'Euler', 'The Rock', 'Blue Devil']


In [166]:
# Sum all the availabilities element-wise, except ignoring if the value is non-numerical
# Therefore, total_avail will contain the total number of people available for each shift

total_avail = None

for df in availabilities:

    df2 = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    if total_avail is None:
        total_avail = df2
    else:
        total_avail += df2

# Add back the times (first column) to the total availability
total_avail = pd.concat([df.iloc[:, 0], total_avail], axis=1)
total_avail.columns = ['Total Available', *total_avail.columns[1:]]

In [167]:
# Now for the actual scheduling

display(total_avail.iloc[-1:, :])


,Total Available,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
38,NIGHT SHIFTS,3,2,2,2,2,2,1


In [168]:
class NightPerson:
    def __init__(self, name, avail_mask):
        self.name = name
        self.avail_mask = avail_mask
        self.working_days = [0] * 7  # What days this person is working on

    def can_work(self, day):
        return self.avail_mask[day]
    
    def num_shifts_worked(self):
        return sum(self.working_days)

    def assign_night_shift(self, day):
        self.working_days[day] = 1

    def unassign_night_shift(self, day):
        self.working_days[day] = 0

    def __repr__(self):
        return f'{self.name}: {self.working_days}'


class NightSchedule:
    def __init__(self, people):
        self.people = people

    def all_nights_worked(self, min_people_night):
        num_days = len(self.people[0].working_days) if self.people else 0

        for day in range(num_days):
            if sum(person.working_days[day] for person in self.people) < min_people_night:
                return False

        return True

    def get_balance(self):
        shifts_worked = [person.num_shifts_worked() for person in self.people]
        return statistics.stdev(shifts_worked) if shifts_worked else 0

    def assign_night_shifts(self, day, balanced_schedules, min_people_night, top=10):
        if day >= 7:
            if self.all_nights_worked(min_people_night):
                balanced_schedules.append(copy.deepcopy(self))
                balanced_schedules.sort(key=lambda x: x.get_balance())

                if len(balanced_schedules) > top:
                    balanced_schedules.pop()
            return

        schedule_copy = copy.deepcopy(self)

        # Get the people who can work on this day
        people_available = [person for person in schedule_copy.people if person.can_work(day)]

        # Get all the combinations of people who can work on this day
        for people in combinations(people_available, min_people_night):
            for person in people:
                person.assign_night_shift(day)

            schedule_copy.assign_night_shifts(day+1, balanced_schedules, min_people_night, top)

            for person in people:
                person.unassign_night_shift(day)


people = []

for p in availabilities:
    name = p.columns[0]
    avail_mask = p.iloc[-1:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).values[0]
    people.append(NightPerson(name, avail_mask))


night_schedule = NightSchedule(people)
balanced_schedules = [] # The top balanced schedules
night_schedule.assign_night_shifts(0, balanced_schedules, min_people_night)

day_map = {'Sunday': 0, 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6}
days_of_week = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

def print_schedule(schedule):
    # Show by day in a dataframe

    # 2*7 df
    # First row is jsut the days of the week
    # Second row is the people working on that day

    df = pd.DataFrame([days_of_week, [''] * 7], columns=[''] * 7)

    for person in schedule.people:
        for day in range(7):
            if person.working_days[day]:
                df.iloc[1, day] += person.name + ', '

    # Display all but first column
    display(df.iloc[:, 1:])

    print('CSV:')
    # CSV is of format: NIGHT SHIFT, name, name, name, ...
    print('NIGHT SHIFT, ', end='')
    string = ''
    for day in range(7):
        string += ', '.join([person.name for person in schedule.people if person.working_days[day]]) + ', '
    print(string[:-2])
    print('\n\n\n')


for idx, schedule in enumerate(balanced_schedules, start=1):
    print(f"Schedule {idx}: Balance: {schedule.get_balance()}")
    print_schedule(schedule)

# If balanced_schedules is empty, then there is no possible schedule using strictly people's ideal availabilities
# Next, we'll randomly keep adding people's 'Q' availabilities until we get a schedule that works, noting which ones we've added
    
def unbalanced_schedule():
    print("No possible schedule using ideal availabilities\n\n")

    class NightScheduleQuestionable(NightSchedule):
        # This class is the same as NightSchedule, except it also keeps track of the people who are working during the questionable days
        def __init__(self, people, people_working_during_questionable, balanced_schedules):
            super().__init__(people)
            self.people_working_during_questionable = people_working_during_questionable
            self.balanced_schedules = balanced_schedules


    # Tabulate all Q availabilities for use
    # i.e. if Michael is available [0, 1, 1, 0, Q, Q, Q], then the dictionary will be {'Michael': ['Thursday', 'Friday', 'Saturday']}

    unsure_map = {} # All Q availabilities

    for avail in availabilities:
        name = avail.columns[0]
        avail_mask = avail.iloc[-1:, 1:]
        # Get indices of all Q's
        unsure = avail_mask[avail_mask == 'Q'].dropna(axis=1).columns.tolist()
        unsure_map[name] = unsure

    unsure_days = [] # ex. ['Thursday', 'Wednesday', 'Saturday', 'Monday', 'Thursday', 'Saturday']
    for name in unsure_map:
        unsure_days += unsure_map[name]

    # Parallel list for who is associated with each day
    q_owners = [] # ex. ['Michael', 'John', 'Michael', 'John', 'Michael', 'John']
    for name in unsure_map:
        q_owners += [name] * len(unsure_map[name])    

    total_Q = len(unsure_days)

    # Now to keep adding people's Q availabilities until we get a schedule that works
    # Start by trying all additions of 1 person's Q availabilities, then 2, etc.
    # Break out of the loop once we find a schedule that works


    sol_found = False
    possible_night_schedules = []

    for num_people in range(1, total_Q+1):

        # Get all combinations of people's Q availabilities for this number of people

        if sol_found:

            # Here we have potentionally a bunch of NightScheduleQuestionable objects that work
            # Each individually has a parameter balanced_schedules of the top balanced schedules
            # We'll now get the top balanced schedules of all of these NightScheduleQuestionable objects, while keeping track of the people_working_during_questionable

            # First get all the balances
            balances = []
            for night_schedule in possible_night_schedules:
                for schedule in night_schedule.balanced_schedules:
                    balances.append(schedule.get_balance())

            # Establish the cutoff for the top balanced schedules (top 5)
            # This will almost certainly produce more than 5 because of ties
            cutoff = sorted(balances)[5] if len(balances) > 5 else 99999

            # Now just go through all the NightScheduleQuestionable objects and print if the balance is above the cutoff
            for night_schedule in possible_night_schedules:
                for schedule in night_schedule.balanced_schedules:
                    if schedule.get_balance() <= cutoff:
                        print(f"Balance: {schedule.get_balance()} Questionable: {schedule.people_working_during_questionable}")
                        print_schedule(schedule)

            return

        for people in combinations(range(total_Q), num_people):


            people_working_during_questionable = {} # To keep track of who we'll be asking to work during the questionable days ex. {'Michael': ['Thursday', 'Saturday'], 'John': ['Wednesday', 'Monday']}
            for idx in people:
                name = q_owners[idx]
                day = unsure_days[idx]
                if name not in people_working_during_questionable:
                    people_working_during_questionable[name] = []
                people_working_during_questionable[name].append(day)


            # As before, now for the people who we are testing to work during the questionable days, we'll expand their availabilities to include the questionable days
            people = []
            for p in availabilities:
                name = p.columns[0]
                avail_mask = p.iloc[-1:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).values[0]

                if name in people_working_during_questionable:
                    # Map 'Sunday' to 0, 'Monday' to 1, etc. from the people_working_during_questionable dictionary
                    # Then set the availability mask to 1 for those days
                    for day in people_working_during_questionable[name]:
                        avail_mask[day_map[day]] = 1

                people.append(NightPerson(name, avail_mask))

                balanced_schedules = [] # The top balanced schedules
                night_schedule = NightScheduleQuestionable(people, people_working_during_questionable, balanced_schedules)
                night_schedule.assign_night_shifts(0, balanced_schedules, min_people_night, top=10)

                if balanced_schedules:
                    possible_night_schedules.append(copy.deepcopy(night_schedule))
                    sol_found = True

    if not sol_found:
        print("No possible schedule using questionable availabilities")

if not balanced_schedules:
    unbalanced_schedule()


No possible schedule using ideal availabilities


Balance: 1.632993161855452 Questionable: {'Barack Obama': ['Saturday']}


,,,,,,
0,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
1,"Barack Obama, The Rock,","Bill Nye, Euler,","Euler, The Rock,","Michael Bryant, Euler,","Barack Obama, Euler,","Barack Obama, Euler,"


CSV:
NIGHT SHIFT, Bill Nye, Blue Devil, Barack Obama, The Rock, Bill Nye, Euler, Euler, The Rock, Michael Bryant, Euler, Barack Obama, Euler, Barack Obama, Euler




Balance: 1.632993161855452 Questionable: {'Bill Nye': ['Saturday']}


,,,,,,
0,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
1,"Barack Obama, The Rock,","Bill Nye, Euler,","Euler, The Rock,","Michael Bryant, Euler,","Barack Obama, Euler,","Bill Nye, Euler,"


CSV:
NIGHT SHIFT, Bill Nye, Blue Devil, Barack Obama, The Rock, Bill Nye, Euler, Euler, The Rock, Michael Bryant, Euler, Barack Obama, Euler, Bill Nye, Euler




Balance: 1.9148542155126762 Questionable: {'Bill Nye': ['Saturday']}


...

CSV:
NIGHT SHIFT, Bill Nye, Blue Devil, Barack Obama, The Rock, Bill Nye, Euler, Euler, The Rock, Michael Bryant, Euler, Barack Obama, Euler, Euler, Blue Devil




Balance: 1.9148542155126762 Questionable: {'Blue Devil': ['Saturday']}


,,,,,,
0,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
1,"Barack Obama, The Rock,","Bill Nye, Euler,","Euler, The Rock,","Michael Bryant, Euler,","Barack Obama, Euler,","Euler, Blue Devil,"


CSV:
NIGHT SHIFT, Bill Nye, Euler, Barack Obama, The Rock, Bill Nye, Euler, Euler, The Rock, Michael Bryant, Euler, Barack Obama, Euler, Euler, Blue Devil




Balance: 1.9148542155126762 Questionable: {'Blue Devil': ['Saturday']}


,,,,,,
0,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
1,"Barack Obama, The Rock,","Bill Nye, Euler,","Euler, The Rock,","Michael Bryant, Euler,","Barack Obama, Euler,","Euler, Blue Devil,"


CSV:
NIGHT SHIFT, Euler, Blue Devil, Barack Obama, The Rock, Bill Nye, Euler, Euler, The Rock, Michael Bryant, Euler, Barack Obama, Euler, Euler, Blue Devil




